<a href="https://colab.research.google.com/github/AngelinaMaverino/ChatBot/blob/copy-googleColab/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot


## Importar librerias

In [ ]:
!pip install sentence-transformers==3.3.0
!pip install --upgrade pinecone
!pip install accelerate==1.1.1
!pip install transformers==4.46.2
!pip install langchain-community==0.3.5
!pip install bitsandbytes==0.44.1
!pip install diffusers==0.27.2
!pip install beautifulsoup4==4.12.3

In [ ]:
from transformers import pipeline

import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

import re

from bs4 import BeautifulSoup

import requests
from bs4 import BeautifulSoup
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone

from pinecone import Pinecone, ServerlessSpec


## Normalizar texto

In [ ]:
def normalize_text(text):
    text = text.lower()
    # Elimina caracteres especiales y multiples espacios en blanco
    text = re.sub(r'[^a-záéíóúñ0-9\s\(\):,\.]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


## Scraping de la web de ort

### Procesamiento de los enlaces dentro de la página

In [ ]:
def extraer_enlaces(soup):
    enlaces = []

    # Obtenemos solo los links que se encuentran en la pagina "central", no en el footer ni header
    central_panel = soup.find('div', id='centralpanel')

    if central_panel:
        for tag in central_panel.find_all('a', href=True):
            enlaces.append(tag['href'])

    return enlaces


También se procesan los diferentes enlaces de la página para obtener más información. Esto no incluye los enlaces del pie de página (footer) ni del encabezado (header), aunque podrían ser incorporados en el futuro.

In [ ]:
def procesar_enlaces(base_url, enlaces):
    documents = []

    for enlace in enlaces:
        if not enlace.startswith('http'):
            enlace = base_url + enlace

        try:
            response = requests.get(enlace)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                docs_from_link = procesar_documento(soup)
                documents.extend(docs_from_link)  # Añadir los documentos procesados a la lista
        except requests.exceptions.RequestException as e:
            print(f"Error al procesar el enlace {enlace}: {e}")

    return documents


### Procesamiento de la pagina

Desarrollamos una función que vincula los títulos con los textos correspondientes, ya que la estructura de la página, caracterizada por títulos y textos dispersos, dificultaba la comprensión del contexto. Con esta función, primero añadimos el título del div al que pertenece el texto y luego el contenido textual. De esta manera, logramos organizar mejor los textos, proporcionando un contexto más claro y una estructura más coherente.

In [ ]:
def procesar_documento(soup):
    documents = []

    for section in soup.find_all('section'):
        current_header = ""
        current_chunk = ""
        current_section_title = ""

        for tag in section.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
            if tag.name.startswith('h'):
                if not current_header:
                    current_header = tag.get_text().strip()
                    current_section_title = tag.get_text().strip()
                else:
                    if current_chunk:
                        documents.append(f"{current_header}: {current_chunk.strip()}")
                        current_chunk = ""
                    current_header = current_section_title + ': ' + tag.get_text().strip()
            else:
                current_chunk += " " + tag.get_text()

        if current_chunk:
            documents.append(f"{current_header}: {current_chunk.strip()}")
            current_chunk = ""

    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
        if tag.find_parent('section') is None:
            documents.append(tag.get_text().strip())

    return documents

### Scraping

In [ ]:
def obtener_documentos(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        documents = procesar_documento(soup)

        enlaces = extraer_enlaces(soup)

        documents_from_links = procesar_enlaces(url, enlaces)

        documents.extend(documents_from_links)

        return documents
    else:
        print(f"Error al acceder a la URL: {response.status_code}")
        return []

In [ ]:
url = "https://fi.ort.edu.uy/ingenieria-en-sistemas"

documents = obtener_documentos(url)

docs = [Document(page_content=normalize_text(doc)) for doc in documents]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

text_chunks = text_splitter.split_documents(docs)

## Creacion de embeddings

Para esto primero se probaron dos modelos diferentes para decidir cual era el mejor modelo de embeddings

### All-MiniLM-L6-v2

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Extraer texto de los chunks
texts = [chunk.page_content for chunk in text_chunks]  # Lista de textos a embebedar

# Crear embeddings usando el modelo con dimensión 384
vectors = embeddings.embed_documents(texts)  # Generar los embeddings para cada chunk

## Base de datos vectorial

### Creacion de la base de datos vectorial

In [ ]:
# Configuracion del API key y creacion de la instancia de Pinecone
pc = Pinecone(api_key="b0f2ee7b-75a9-4c4e-a7fe-f94deb2e984b")

index_name = "ort-5"

# Verificar si el indice ya existe
existing_indexes = pc.list_indexes().names()
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


# Conectar al indice
index = pc.Index(index_name)

## Almacenamiento en base de datos

In [ ]:
batch_size = 1000

# Dividir los vectores en lotes y hacer upsert de cada uno
for i in range(0, len(vectors), batch_size):
    batch_vectors = vectors[i:i + batch_size]
    batch_texts = texts[i:i + batch_size]

    upserts = [(str(i + j), vector, {"content": text})
            for j, (vector, text) in enumerate(zip(batch_vectors, batch_texts))]

    # Subir el lote a Pinecone
    index.upsert(upserts)

print(f"Vector store creado con éxito en Pinecone con {len(text_chunks)} embeddings.")

## Modelo de generacion de textos

In [ ]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [ ]:
# Crear el generador de texto
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
def get_relevant_context(question, top_k=25):
    # Generar embedding de la pregunta
    question_embedding = embeddings.embed_documents([question])[0]

    # Utilizamos Pinecone para realizar busqueda por similitud, que fue seteada cuando creamos la bd, utilizando la metrica dotproduct, que fue la que mejores respuestas nos dio
    results = index.query(vector=question_embedding, top_k=top_k, include_metadata=True)

    context = [match['metadata']['content'] for match in results['matches']]

    return context

In [ ]:
def ask_bot(question):

    context = get_relevant_context(question)

    if not context:
        return "No se encontraron documentos relevantes para la pregunta."

    prompt = (
        f"Usa solo la siguiente información para responder de forma precisa:\n\n"
        f"Contexto:\n{context}\n\n"
        f"Pregunta: {question}\n\n"
        f"Responde de manera precisa y breve:"
    )

    # Generar la respuesta usando el contexto
    response = generator(
        prompt, max_new_tokens=500, truncation=True
    )[0]['generated_text']

    generated_answer = response.split("Responde de manera precisa y breve:")[-1].strip()

    return generated_answer

## Preguntas de prueba

In [ ]:
pregunta = normalize_text("¿Cuál es la duración de años?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cuáles son los horarios de clase matutino?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿horarios de clase para semestre nocturno?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿horarios de clase vespertino?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno matutino del semestre 1 al 4?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno vespertino del semestre 5 en adelante?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno nocturno?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que son las Áreas de profundización?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("Dame info sobre el area de profundizacion de Inteligencia Artificial y Analítica de Datos")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)


In [ ]:
pregunta = normalize_text("¿cuando se informan los valores de las cuotas?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cual es el Perfil de los graduados de ingeniería en sistemas?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Quiénes pueden cursar el CerIA?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)


In [ ]:
pregunta =  normalize_text("¿El CerIA es un título de postgrado?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿A quienes esta dirigido el Servicio de orientación laboral?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que se hace en el Servicio de orientación laboral?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cuando la Facultad de Ingeniería recibió por parte de la Comisión ad hoc de la Acreditación ARCU-SUR?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que cargos ocupan los graduados?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que es el Taller de Nivelación de Conocimientos de Matemática?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)